# **SET-UP AND DEPENDENCIES**

In [ ]:
from google.colab import drive
# mount your google drive

Mounted at /content/drive


In [ ]:
path = your path

text_access = "text"

# Read only the specified text column
df_discharge_text = pd.read_csv(path, usecols=[text_access], compression='gzip')

In [41]:
# Keep only 1K records for computational efficiency.
df_discharge_text = df_discharge_text.iloc[:1000].reset_index(drop=True)
df_discharge_text.head(2)

text
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nWorsening ABD distension and pain \n \nMajor Surgical or Invasive Procedure:\nParacentesis\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with

# **TEXT CLEAN-UP**

In [8]:
# Create an empty data frame for results.
df_extract = df_discharge_text.drop("text", axis = 1)

In [9]:
# User Defined Function to extract Chief Complaint: (everything after "Chief Complaint:" until the end of the line) and add it as a column to the original dataframe.

def extract_complaint(pattern, df_discharge_text, column_name="text"):
    """
    Extracts matches based on a pre-defined regex pattern from a specified column in the dataframe.

    Parameters:
        pattern (str): The regular expression pattern to use for extraction: r'Chief Complaint:\s*(.*)'
        df (pd.DataFrame): The DataFrame containing the text column: df_discharge_text.
        column_name (str): The name of the column to search, "text_access".

    Returns:
        pd.Series: A Series containing the extracted matches which will be added as a new column to the original dataframe.
    """
    # applying the pre-defined regex to each row of the target column
    return df_discharge_text[column_name].apply(lambda text: re.search(pattern, str(text)).group(1) if re.search(pattern, str(text)) else None)

# df[column_name] → Selects the target column ("text_access" in df_discharge_text).
# .apply(lambda text: ...) → Applies a function to each row in that column.
# str(text) → Converts the value to a string (in case it contains NaN or numbers).
# re.search(pattern, str(text)) → Searches for the regex pattern inside each row’s text:
# If no match is found, re.search() returns None.
# If a match is found, .group(1) extracts the first capturing group from the regex.
# if re.search(pattern, str(text)) else None → Ensures that if no match is found, it returns None instead of causing an error.

# Define regex pattern
pattern = r'Chief Complaint:\s*(.*)'

# Apply function to extract chief complaints
df_extract["Chief Complaint"] = extract_complaint(pattern, df_discharge_text, "text")

# Display results
# df_extract

In [10]:
# User Defined Function to extract Diagnoses: (multi-line after "Discharge Diagnoses" until break or Discharge Condition:) and add it as a column to the original dataframe.

def extract_all_dz(pattern, df_discharge_text, column_name="text"):
    """
    Extracts matches based on a pre-defined regex pattern from a specified column in the dataframe.

    Parameters:
        pattern (str): The regular expression pattern to use for extraction:
        df (pd.DataFrame): The DataFrame containing the text column: df_discharge_text.
        column_name (str): The name of the column to search, "text_access".

    Returns:
        pd.Series: A Series containing the extracted matches which will be added as a new column to the original dataframe.
    """
    # applying the pre-defined regex to each row of the target column
    return df_discharge_text[column_name].apply(lambda text: re.search(pattern, str(text)).group(1) if re.search(pattern, str(text)) else None)


# Define regex pattern
pattern = pattern = r'(?s)Discharge Diagnosis:\s*(.+?)(Discharge Condition:|\Z)'

# \n\s*\n stops at the first blank line (double newline) to prevent capturing unwanted sections.
# \Z ensures it captures everything until the end if no blank line is found.

# Apply function to extract diagnosis
df_extract["All Dz"] = extract_all_dz(pattern, df_discharge_text, "text")

# Display results
# df_extract

In [11]:
# Since we captured all non-white space following Discharge Diagnosis, we need to remove unnecessary characters and words.
# First, find and remove special characters, to include creative patterns like "====" or "---" and digits

df_extract["All Dx"] = df_extract["All Dz"].str.replace(r'(\s*\n\s*|- |-|={3,}|-{3,}|_{3,}|#|\d+)', ' ', regex=True).str.strip()


# Second, remove "Primary diagnosis:" and "Secondary diagnosis:"
pattern = r'(?i)Primary diagnosis:|Secondary diagnosis:|Primary diagnoses:|Primary Diagnosis|Secondary Diagnosis|Secondary diagnoses:|PRIMARY DIAGNOSIS|SECONDARY DIAGNOSIS|PRIMARY DIAGNOSES|SECONDARY DIAGNOSES'

df_extract["All Dx"] = df_extract["All Dx"].str.replace(pattern, '', regex=True).str.strip()
# df_extract

In [39]:
# User Defined Function to extract History of Present Illness: (everything after "History of Present Illness:" until break) and add it as a column to the original dataframe.

def extract_present_illness(pattern, df_discharge_text, column_name="text"):
    """
    Extracts matches based on a pre-defined regex pattern from a specified column in the dataframe.

    Parameters:
        pattern (str): The regular expression pattern to use for extraction: r'History of Present Illness:\s*(.*)'
        df (pd.DataFrame): The DataFrame containing the text column: df_discharge_text.
        column_name (str): The name of the column to search, "text".

    Returns:
        pd.Series: A Series containing the extracted matches which will be added as a new column to the original dataframe.
    """
    # applying the pre-defined regex to each row of the target column
    return df_discharge_text[column_name].apply(lambda text: re.search(pattern, str(text)).group(1) if re.search(pattern, str(text)) else None)


# Define regex pattern
pattern = pattern = r'(?s)History of Present Illness:\s*(.+?)(Past Medical History:|\Z)'

# \n\s*\n stops at the first blank line (double newline) to prevent capturing unwanted sections.
# \Z ensures it captures everything until the end if no blank line is found.

# Apply function to extract history of present illness
df_extract["Present Illness History Extracted"] = extract_present_illness(pattern, df_discharge_text, "text")
# df_extract.head(2)

In [13]:
# Since we captured all text as is for History of Present Illness, we need to remove unnecessary characters.
# Find and remove special characters and new lines.

df_extract["Present Illness History"] = df_extract["Present Illness History Extracted"].str.replace(r'(\s*\n\s*|_{3,}|={3,}|\+)', ' ', regex=True).str.strip()
# df_extract.iloc[4]

In [14]:
df_extract.drop(["Present Illness History Extracted", "All Dz"], axis = 1, inplace = True)

In [15]:
df_extract.head(2)

,Chief Complaint,All Dx,Present Illness History
0,Worsening ABD distension and pain,Ascites from Portal HTN,"HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone weeks ago, because she feels like ""they don't do anything"" and that she ""doesn't want to put more chemicals in her."" She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing her teeth in recent weeks. she denies easy bruising, melena, BRBPR, hemetesis, hemoptysis, or hematuria. Because of her abd pain, she went to OSH ED and was transferred to for further care. Per ED report, pt has brief period of confusion - she did not recall the ultrasound or bloodwork at osh. She denies recent drug use or alcohol use. She denies feeling confused, but reports that she is forgetful at times. In the ED, initial vitals were 98.4 70 106/63 16 97%RA Labs notable for ALT/AST/AP : , Tbili1.6, WBC 5K, platelet 77, INR 1.6"
1,abdominal fullness and discomfort,"Primary: diuretic refractory ascites Secondary: HCV cirrhosis, HIV, hyponatremia, COPD","with HIV on HAART, COPD, HCV cirrhosis complicated by ascites and HE admitted with abdominal distention and pain. She was admitted to for the same symptoms recently and had 3L fluid removed (no SBP) three days ago and felt better. Since discharge, her abdomen has become increasingly distended with pain. This feels similar to prior episodes of ascites. Her diuretics were recently decreased on due to worsening hyponatremia 128 and hyperkalemia 5.1. Patient states she has been compliant with her HIV and diuretic medications but never filled out the lactulose prescription. She states she has had BMs daily at home. She has had some visual hallucinations and forgetfulness. Her appetite has been poor. In the ED, initial vitals were 98.9 88 116/88 18 97% RA. CBC near baseline, INR 1.4, Na 125, Cr 0.6. AST and ALT mildly above baseline 182 and 126 and albumin 2.8. Diagnostic para with 225 WBC, 7% PMN, total protein 0.3. UA with few bact, 6 WBC, mod leuk, neg nitr, but contaminated with 6 epi. CXR clear. RUQ US with no PV thrombus, moderate ascites. She was given ondansetron 4mg IV and morphine 2.5mg IV x1 in the ED. On the floor, she is feeling improved but still has abdominal distention and discomfort. ROS: Abdominal distention and pain. No black/bloody stools. No pain or swelling. No fevers or chills. Denies chest pain, nausea, vomiting. No dysuria or frequency."


# **PRE-PROCESSING FOR LDA**

Common pre-processing for LDA includes:
1. Cleaning text of extraneous characters (already done above).
2. Tokenization.
  Tokenization of medical texts requires special attention due to unique and complex characteristics of medical content, among which we find: special structures like abbreviations; compound terms like "chronic kidney disease which must remain fused / used as single tokens; negation found in false fused expressions such as "patient denies" or "no chest pain" which must not be fragmented.
  In light of these challenges, the recommended approach is to use the pre-trained small clinical model from spaCy. However, upon several tests, a modified spaCy tokenizer is the final recommended approach. As a special rule, we will ensure that 1) numerical values and % sign are not seperated; 2) negative contractions like "don't" and "can't" are not separated; mutli-character separated by / are preserved.
3. For clinical text: expansion of clinical abbreviations.
  As there are numerous abbreviations in clinical text, the next step, lowercasing, may jeopardize meaningful topic extraction. Thus, we will extend clinical abbreviations based on what's presented in the notes; a more advanced approach is applying tools like QuickUMLS, MetaMap, or clinical BERT models. However, pre-trained models may over-normalize the text which will impact the quality of topic modeling, espcially if semantic accuracy is sufficient to approximate (not to reach 100% match).  

In [17]:
# Using nltk modified tokenizer with the following rules:
# \d+%            keep numbers and % signs together like 100%
# \w+/\w+(?:/\w+)* keep slash-separated like f/v/c together
# \w+(?:-\w+)+    keep hyphenated like Na-restricted together
# \w+'\w+         keep contractions like don't together
# \w+             normal words
# [^\w\s]         keep punctuation (for now)
token_pattern = r"\d+%|\w+/\w+(?:/\w+)*|\w+(?:-\w+)+|\w+'\w+|\w+|[^\w\s]"

nltk_tokenizer = RegexpTokenizer(token_pattern)

def tokenize_text_nltk(text):
    if pd.isnull(text):
        return []
    return nltk_tokenizer.tokenize(str(text))

df_extract["tokens"] = df_extract["Present Illness History"].apply(tokenize_text_nltk)
df_extract.head(2)

,Chief Complaint,All Dx,Present Illness History,tokens
0,Worsening ABD distension and pain,Ascites from Portal HTN,"HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone weeks ago, because she feels like ""they don't do anything"" and that she ""doesn't want to put more chemicals in her."" She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing her teeth in recent weeks. she denies easy bruising, melena, BRBPR, hemetesis, hemoptysis, or hematuria. Because of her abd pain, she went to OSH ED and was transferred to for further care. Per ED report, pt has brief period of confusion - she did not recall the ultrasound or bloodwork at osh. She denies recent drug use or alcohol use. She denies feeling confused, but reports that she is forgetful at times. In the ED, initial vitals were 98.4 70 106/63 16 97%RA Labs notable for ALT/AST/AP : , Tbili1.6, WBC 5K, platelet 77, INR 1.6","[HCV, cirrhosis, c/b, ascites, ,, hiv, on, ART, ,, h/o, IVDU, ,, COPD, ,, bioplar, ,, PTSD, ,, presented, from, OSH, ED, with, worsening, abd, distension, over, past, week, ., Pt, reports, self-discontinuing, lasix, and, spirnolactone, weeks, ago, ,, because, she, feels, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemicals, in, her, ., "", She, does, not, follow, Na-restricted, diets, ., In, the, past, week, ,, she, notes, that, she, has, been, having, worsening, abd, distension, and, discomfort, ., She, denies, edema, ,, or, SOB, ,, or, orthopnea, ., She, denies, ...]"
1,abdominal fullness and discomfort,"Primary: diuretic refractory ascites Secondary: HCV cirrhosis, HIV, hyponatremia, COPD","with HIV on HAART, COPD, HCV cirrhosis complicated by ascites and HE admitted with abdominal distention and pain. She was admitted to for the same symptoms recently and had 3L fluid removed (no SBP) three days ago and felt better. Since discharge, her abdomen has become increasingly distended with pain. This feels similar to prior episodes of ascites. Her diuretics were recently decreased on due to worsening hyponatremia 128 and hyperkalemia 5.1. Patient states she has been compliant with her HIV and diuretic medications but never filled out the lactulose prescription. She states she has had BMs daily at home. She has had some visual hallucinations and forgetfulness. Her appetite has been poor. In the ED, initial vitals were 98.9 88 116/88 18 97% RA. CBC near baseline, INR 1.4, Na 125, Cr 0.6. AST and ALT mildly above baseline 182 and 126 and albumin 2.8. Diagnostic para with 225 WBC, 7% PMN, total protein 0.3. UA with few bact, 6 WBC, mod leuk, neg nitr, but contaminated with 6 epi. CXR clear. RUQ US with no PV thrombus, moderate ascites. She was given ondansetron 4mg IV and morphine 2.5mg IV x1 in the ED. On the floor, she is feeling improved but still has abdominal distention and discomfort. ROS: Abdominal distention and pain. No black/bloody stools. No pain or swelling. No fevers or chills. Denies chest pain, nausea, vomiting. No dysuria or frequency.","[with, HIV, on, HAART, ,, COPD, ,, HCV, cirrhosis, complicated, by, ascites, and, HE, admitted, with, abdominal, distention, and, pain, ., She, was, admitted, to, for, the, same, symptoms, recently, and, had, 3L, fluid, removed, (, no, SBP, ), three, days, ago, and, felt, better, ., Since, discharge, ,, her, abdomen, has, become, increasingly, distended, with, pain, ., This, feels, similar, to, prior, episodes, of, ascites, ., Her, diuretics, were, rec

In [18]:
# Checking abbreviations by extracting most likely combinations and formats: capitalized, used at least twicw, 2 - 5 letters.
def extract_and_merge_abbreviations(token_lists, min_freq=2):
    # Flatten list of token lists
    all_tokens = [token for tokens in token_lists for token in tokens]

    # Candidate abbreviations (all-caps, 2–5 letters)
    candidate_abbrs = [
        tok for tok in all_tokens
        if tok.isalpha() and tok.isupper() and 2 <= len(tok) <= 5
    ]

    # Slash-based (e.g., "N/V/D")
    slash_abbrs = []
    for tok in all_tokens:
        if '/' in tok and re.match(r'^([A-Za-z]{1,5}/)+[A-Za-z]{1,5}$', tok):
            slash_abbrs.extend(part.upper() for part in tok.split('/'))

    all_abbrs = candidate_abbrs + slash_abbrs
    abbr_counter = Counter(all_abbrs)
    # Return the Counter object to make the results available
    return abbr_counter # Added this line to return the counter

identified_abbrs = extract_and_merge_abbreviations(df_extract["tokens"])
identified_abbrs.most_common(10)

[('ED', 1091),
 ('P', 486),
 ('IV', 457),
 ('S', 457),
 ('RA', 448),
 ('CT', 346),
 ('O', 272),
 ('WBC', 256),
 ('BP', 242),
 ('PO', 241)]

In [19]:
 # Top abbreviations expansions as a sorted dictionary.

abbr_dict_manual = dict(sorted({
    "A": "artery",
    "ALT": "alanine aminotransferase",
    "AM": "morning",
    "ASA": "aspirin",
    "AST": "aspartate aminotransferase",
    "BID": "twice a day",
    "BM": "bowel movement",
    "BNP": "brain natriuretic peptide",
    "BRBPR": "bright red blood per rectum",
    "BP": "blood pressure",
    "BUN": "blood urea nitrogen",
    "CABG": "coronary artery bypass graft",
    "CAD": "coronary artery disease",
    "CBC": "complete blood count",
    "CHF": "congestive heart failure",
    "CKD": "chronic kidney disease",
    "COPD": "chronic obstructive pulmonary disease",
    "CP": "chest pain",
    "CT": "computed tomography",
    "CTA": "computed tomography angiography",
    "CXR": "chest x-ray",
    "DM": "diabetes mellitus",
    "DOE": "dyspnea on exertion",
    "DVT": "deep vein thrombosis",
    "ECG": "electrocardiogram",
    "ED": "emergency department",
    "EF": "ejection fraction",
    "EGD": "esophagogastroduodenoscopy",
    "EKG": "electrocardiogram",
    "EMS": "emergency medical services",
    "ER": "emergency room",
    "ERCP": "endoscopic retrograde cholangiopancreatography",
    "ESRD": "end-stage renal disease",
    "GI": "gastrointestinal",
    "HA": "headache",
    "HAART": "highly active antiretroviral therapy",
    "HD": "hemodialysis",
    "HE": "hepatic encephalopathy",
    "H/H": "hemoglobin and hematocrit",
    "HCV": "hepatitis C virus",
    "HIV": "human immunodeficiency virus",
    "HLD": "hyperlipidemia",
    "HPI": "history of present illness",
    "HR": "heart rate",
    "HTN": "hypertension",
    "ICU": "intensive care unit",
    "INR": "international normalized ratio",
    "IV": "intravenous",
    "IVDU": "intravenous drug use",
    "IVF": "intravenous fluids",
    "LAD": "left anterior descending artery",
    "LOC": "loss of consciousness",
    "LLE": "left lower extremity",
    "MICU": "medical intensive care unit",
    "MI": "myocardial infarction",
    "MRI": "magnetic resonance imaging",
    "Na": "sodium",
    "NC": "nasal cannula",
    "NG": "nasogastric",
    "NS": "normal saline",
    "O2": "oxygen",
    "OSH": "outside hospital",
    "PE": "pulmonary embolism",
    "PCP": "primary care provider",
    "PNA": "pneumonia",
    "PMH": "past medical history",
    "PND": "paroxysmal nocturnal dyspnea",
    "PO": "by mouth",
    "PTSD": "post-traumatic stress disorder",
    "RA": "room air",
    "ROS": "review of systems",
    "RR": "respiratory rate",
    "RUQ": "right upper quadrant",
    "SBP": "spontaneous bacterial peritonitis",
    "SI": "suicidal ideation",
    "SOB": "shortness of breath",
    "ST": "sinus tachycardia",
    "TIA": "transient ischemic attack",
    "UA": "urinalysis",
    "US": "ultrasound",
    "UTI": "urinary tract infection",
    "VS": "vital signs",
    "WBC": "white blood cell",
    "WNL": "within normal limits",
    "ART": "antiretroviral therapy",
    "d/c": "discharge or discontinue",
    "f/c/n/v": "fever, chills, nausea, and vomiting"
}.items()))
# abbr_dict_manual

In [20]:
# Expand abbreviations found in top abbreviations list.
def expand_abbreviations(token_list, abbr_dict):
    return [
        abbr_dict[token] if token in abbr_dict else token
        for token in token_list
    ]

# Apply row by row to the tokenized text.
df_extract["tokens_expanded"] = df_extract["tokens"].apply(
    lambda tokens: expand_abbreviations(tokens, abbr_dict_manual)
)

df_extract.head(2)

,Chief Complaint,All Dx,Present Illness History,tokens,tokens_expanded
0,Worsening ABD distension and pain,Ascites from Portal HTN,"HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone weeks ago, because she feels like ""they don't do anything"" and that she ""doesn't want to put more chemicals in her."" She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing her teeth in recent weeks. she denies easy bruising, melena, BRBPR, hemetesis, hemoptysis, or hematuria. Because of her abd pain, she went to OSH ED and was transferred to for further care. Per ED report, pt has brief period of confusion - she did not recall the ultrasound or bloodwork at osh. She denies recent drug use or alcohol use. She denies feeling confused, but reports that she is forgetful at times. In the ED, initial vitals were 98.4 70 106/63 16 97%RA Labs notable for ALT/AST/AP : , Tbili1.6, WBC 5K, platelet 77, INR 1.6","[HCV, cirrhosis, c/b, ascites, ,, hiv, on, ART, ,, h/o, IVDU, ,, COPD, ,, bioplar, ,, PTSD, ,, presented, from, OSH, ED, with, worsening, abd, distension, over, past, week, ., Pt, reports, self-discontinuing, lasix, and, spirnolactone, weeks, ago, ,, because, she, feels, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemicals, in, her, ., "", She, does, not, follow, Na-restricted, diets, ., In, the, past, week, ,, she, notes, that, she, has, been, having, worsening, abd, distension, and, discomfort, ., She, denies, edema, ,, or, SOB, ,, or, orthopnea, ., She, denies, ...]","[hepatitis C virus, cirrhosis, c/b, ascites, ,, hiv, on, antiretroviral therapy, ,, h/o, intravenous drug use, ,, chronic obstructive pulmonary disease, ,, bioplar, ,, post-traumatic stress disorder, ,, presented, from, outside hospital, emergency department, with, worsening, abd, distension, over, past, week, ., Pt, reports, self-discontinuing, lasix, and, spirnolactone, weeks, ago, ,, because, she, feels, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemicals, in, her, ., "", She, does, not, follow, Na-restricted, diets, ., In, the, past, week, ,, she, notes, that, she, has, been, having, worsening, abd, distension, and, discomfort, ., She, denies, edema, ,, or, shortness of breath, ,, or, orthopnea, ., She, denies, ...]"
1,abdominal fullness and discomfort,"Primary: diuretic refractory ascites Secondary: HCV cirrhosis, HIV, hyponatremia, COPD","with HIV on HAART, COPD, HCV cirrhosis complicated by ascites and HE admitted with abdominal distention and pain. She was admitted to for the same symptoms recently and had 3L fluid removed (no SBP) three days ago and felt better. Since discharge, her abdomen has become increasingly distended with pain. This feels similar to prior episodes of ascites. Her diuretics were recently decreased on due to worsening hyponatremia 128 and hyperkalemia 5.1. Patient states she has been compliant with her HIV and diuretic medications but never filled out the lactulose prescription. She states she has had BMs daily at home. She has had some visual hallucinations and forgetfulness. Her appetite has been poor. In the ED, initial vitals were 98.9 88 116/88 18 97% RA. CBC near baseline, INR 1.4, Na 125, Cr 0.6. AST and ALT mildly above baseline 182 and 126 and albumin 2.8. Diagnostic para with 225 WBC, 7% PMN, total protein 0.3. UA with few bact, 6 WBC, mod leuk, neg nitr, but contaminated with 6 epi. CXR clear. RUQ US with no PV thrombus, m

In [21]:
# Apply lemmatization with prior lowercasing. Remaining abbreviations will be lowercased.
# Split hyphenated tokens and multi-word tokens and then rejoin them.
# Use POS tagging for more precision.
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):
    """
    Map POS tags from nltk.pos_tag to WordNet POS tags for lemmatization.

    Parameters:
        treebank_tag (str): The POS tag from nltk.pos_tag.

    Returns:
        str: The corresponding WordNet POS tag.
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Defaults to noun if not found


def preprocess_for_lda(token_list):
    lemmatized_tokens = []

    for token in token_list:
        token = token.lower()

        # Step 1: Handle multi-word phrases
        if " " in token:
            words = nltk.word_tokenize(token)
            tagged = nltk.pos_tag(words)
            lemmatized = [lemmatizer.lemmatize(w, get_wordnet_pos(pos)) for w, pos in tagged]
            lemmatized_token = " ".join(lemmatized)

        # Step 2: Handle hyphenated words (e.g., "self-discontinuing")
        elif "-" in token:
            parts = token.split("-")
            tagged = nltk.pos_tag(parts)
            lemmatized = [lemmatizer.lemmatize(p, get_wordnet_pos(pos)) for p, pos in tagged]
            lemmatized_token = "-".join(lemmatized)

        # Step 3: Single word
        else:
            pos = get_wordnet_pos(nltk.pos_tag([token])[0][1])
            lemmatized_token = lemmatizer.lemmatize(token, pos)

        lemmatized_tokens.append(lemmatized_token)

    return lemmatized_tokens

# Apply to your DataFrame
df_extract["tokens_lemmatized"] = df_extract["tokens_expanded"].apply(preprocess_for_lda)

df_extract.head(2)

,Chief Complaint,All Dx,Present Illness History,tokens,tokens_expanded,tokens_lemmatized
0,Worsening ABD distension and pain,Ascites from Portal HTN,"HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone weeks ago, because she feels like ""they don't do anything"" and that she ""doesn't want to put more chemicals in her."" She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing her teeth in recent weeks. she denies easy bruising, melena, BRBPR, hemetesis, hemoptysis, or hematuria. Because of her abd pain, she went to OSH ED and was transferred to for further care. Per ED report, pt has brief period of confusion - she did not recall the ultrasound or bloodwork at osh. She denies recent drug use or alcohol use. She denies feeling confused, but reports that she is forgetful at times. In the ED, initial vitals were 98.4 70 106/63 16 97%RA Labs notable for ALT/AST/AP : , Tbili1.6, WBC 5K, platelet 77, INR 1.6","[HCV, cirrhosis, c/b, ascites, ,, hiv, on, ART, ,, h/o, IVDU, ,, COPD, ,, bioplar, ,, PTSD, ,, presented, from, OSH, ED, with, worsening, abd, distension, over, past, week, ., Pt, reports, self-discontinuing, lasix, and, spirnolactone, weeks, ago, ,, because, she, feels, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemicals, in, her, ., "", She, does, not, follow, Na-restricted, diets, ., In, the, past, week, ,, she, notes, that, she, has, been, having, worsening, abd, distension, and, discomfort, ., She, denies, edema, ,, or, SOB, ,, or, orthopnea, ., She, denies, ...]","[hepatitis C virus, cirrhosis, c/b, ascites, ,, hiv, on, antiretroviral therapy, ,, h/o, intravenous drug use, ,, chronic obstructive pulmonary disease, ,, bioplar, ,, post-traumatic stress disorder, ,, presented, from, outside hospital, emergency department, with, worsening, abd, distension, over, past, week, ., Pt, reports, self-discontinuing, lasix, and, spirnolactone, weeks, ago, ,, because, she, feels, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemicals, in, her, ., "", She, does, not, follow, Na-restricted, diets, ., In, the, past, week, ,, she, notes, that, she, has, been, having, worsening, abd, distension, and, discomfort, ., She, denies, edema, ,, or, shortness of breath, ,, or, orthopnea, ., She, denies, ...]","[hepatitis c virus, cirrhosis, c/b, ascites, ,, hiv, on, antiretroviral therapy, ,, h/o, intravenous drug use, ,, chronic obstructive pulmonary disease, ,, bioplar, ,, post-traumatic stress disorder, ,, present, from, outside hospital, emergency department, with, worsen, abd, distension, over, past, week, ., pt, report, self-discontinue, lasix, and, spirnolactone, week, ago, ,, because, she, feel, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemical, in, her, ., "", she, do, not, follow, na-restrict, diet, ., in, the, past, week, ,, she, note, that, she, have, be, have, worsen, abd, distension, and, discomfort, ., she, denies, edema, ,, or, shortness of breath, ,, or, orthopnea, ., she, denies, ...]"
1,abdominal fullness and discomfort,"Primary: diuretic refractory ascites Secondary: HCV cirrhosis, HIV, hyponatremia, COPD","with HIV on HAART, COPD, HCV cirrhosis complicated by ascites and HE admitted with abdominal distention and pain. She was admitted to for the same symptoms recently and had 3L fluid removed (no SBP) three days ago and felt better. Since discharge, her abdomen has

In [25]:
# Create a set of all punctuation characters for fast look-up
punct_set = set(string.punctuation)

# Filter out punctuation tokens from each note and save results in a new column.
df_extract["tokens_final"] = df_extract["tokens_lemmatized"].apply(
    lambda tokens: [tok for tok in tokens if tok not in punct_set]
)

df_extract.head(2)

,Chief Complaint,All Dx,Present Illness History,tokens,tokens_expanded,tokens_lemmatized,tokens_final
0,Worsening ABD distension and pain,Ascites from Portal HTN,"HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone weeks ago, because she feels like ""they don't do anything"" and that she ""doesn't want to put more chemicals in her."" She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing her teeth in recent weeks. she denies easy bruising, melena, BRBPR, hemetesis, hemoptysis, or hematuria. Because of her abd pain, she went to OSH ED and was transferred to for further care. Per ED report, pt has brief period of confusion - she did not recall the ultrasound or bloodwork at osh. She denies recent drug use or alcohol use. She denies feeling confused, but reports that she is forgetful at times. In the ED, initial vitals were 98.4 70 106/63 16 97%RA Labs notable for ALT/AST/AP : , Tbili1.6, WBC 5K, platelet 77, INR 1.6","[HCV, cirrhosis, c/b, ascites, ,, hiv, on, ART, ,, h/o, IVDU, ,, COPD, ,, bioplar, ,, PTSD, ,, presented, from, OSH, ED, with, worsening, abd, distension, over, past, week, ., Pt, reports, self-discontinuing, lasix, and, spirnolactone, weeks, ago, ,, because, she, feels, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemicals, in, her, ., "", She, does, not, follow, Na-restricted, diets, ., In, the, past, week, ,, she, notes, that, she, has, been, having, worsening, abd, distension, and, discomfort, ., She, denies, edema, ,, or, SOB, ,, or, orthopnea, ., She, denies, ...]","[hepatitis C virus, cirrhosis, c/b, ascites, ,, hiv, on, antiretroviral therapy, ,, h/o, intravenous drug use, ,, chronic obstructive pulmonary disease, ,, bioplar, ,, post-traumatic stress disorder, ,, presented, from, outside hospital, emergency department, with, worsening, abd, distension, over, past, week, ., Pt, reports, self-discontinuing, lasix, and, spirnolactone, weeks, ago, ,, because, she, feels, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemicals, in, her, ., "", She, does, not, follow, Na-restricted, diets, ., In, the, past, week, ,, she, notes, that, she, has, been, having, worsening, abd, distension, and, discomfort, ., She, denies, edema, ,, or, shortness of breath, ,, or, orthopnea, ., She, denies, ...]","[hepatitis c virus, cirrhosis, c/b, ascites, ,, hiv, on, antiretroviral therapy, ,, h/o, intravenous drug use, ,, chronic obstructive pulmonary disease, ,, bioplar, ,, post-traumatic stress disorder, ,, present, from, outside hospital, emergency department, with, worsen, abd, distension, over, past, week, ., pt, report, self-discontinue, lasix, and, spirnolactone, week, ago, ,, because, she, feel, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemical, in, her, ., "", she, do, not, follow, na-restrict, diet, ., in, the, past, week, ,, she, note, that, she, have, be, have, worsen, abd, distension, and, discomfort, ., she, denies, edema, ,, or, shortness of breath, ,, or, orthopnea, ., she, denies, ...]","[hepatitis c virus, cirrhosis, c/b, ascites, hiv, on, antiretroviral therapy, h/o, intravenous drug use, chronic obstructive pulmonary disease, bioplar, post-traumatic stress disorder, present, from, outside hospital, emergency department, with, worsen, abd, distension, over, past, week, pt, report, self-discontinue, lasix, and, spirnolactone, week, ago, because, she, feel

In [34]:
# Remove stopwords (common English stopwords)
# Create a set for fast lookup
stop_words = set(stopwords.words('english'))
df_extract["tokens_final"] = df_extract["tokens_final"].apply(
    lambda tokens: [tok for tok in tokens if tok not in stop_words]
)
df_extract.head(2)

,Chief Complaint,All Dx,Present Illness History,tokens,tokens_expanded,tokens_lemmatized,tokens_final
0,Worsening ABD distension and pain,Ascites from Portal HTN,"HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone weeks ago, because she feels like ""they don't do anything"" and that she ""doesn't want to put more chemicals in her."" She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing her teeth in recent weeks. she denies easy bruising, melena, BRBPR, hemetesis, hemoptysis, or hematuria. Because of her abd pain, she went to OSH ED and was transferred to for further care. Per ED report, pt has brief period of confusion - she did not recall the ultrasound or bloodwork at osh. She denies recent drug use or alcohol use. She denies feeling confused, but reports that she is forgetful at times. In the ED, initial vitals were 98.4 70 106/63 16 97%RA Labs notable for ALT/AST/AP : , Tbili1.6, WBC 5K, platelet 77, INR 1.6","[HCV, cirrhosis, c/b, ascites, ,, hiv, on, ART, ,, h/o, IVDU, ,, COPD, ,, bioplar, ,, PTSD, ,, presented, from, OSH, ED, with, worsening, abd, distension, over, past, week, ., Pt, reports, self-discontinuing, lasix, and, spirnolactone, weeks, ago, ,, because, she, feels, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemicals, in, her, ., "", She, does, not, follow, Na-restricted, diets, ., In, the, past, week, ,, she, notes, that, she, has, been, having, worsening, abd, distension, and, discomfort, ., She, denies, edema, ,, or, SOB, ,, or, orthopnea, ., She, denies, ...]","[hepatitis C virus, cirrhosis, c/b, ascites, ,, hiv, on, antiretroviral therapy, ,, h/o, intravenous drug use, ,, chronic obstructive pulmonary disease, ,, bioplar, ,, post-traumatic stress disorder, ,, presented, from, outside hospital, emergency department, with, worsening, abd, distension, over, past, week, ., Pt, reports, self-discontinuing, lasix, and, spirnolactone, weeks, ago, ,, because, she, feels, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemicals, in, her, ., "", She, does, not, follow, Na-restricted, diets, ., In, the, past, week, ,, she, notes, that, she, has, been, having, worsening, abd, distension, and, discomfort, ., She, denies, edema, ,, or, shortness of breath, ,, or, orthopnea, ., She, denies, ...]","[hepatitis c virus, cirrhosis, c/b, ascites, ,, hiv, on, antiretroviral therapy, ,, h/o, intravenous drug use, ,, chronic obstructive pulmonary disease, ,, bioplar, ,, post-traumatic stress disorder, ,, present, from, outside hospital, emergency department, with, worsen, abd, distension, over, past, week, ., pt, report, self-discontinue, lasix, and, spirnolactone, week, ago, ,, because, she, feel, like, "", they, don't, do, anything, "", and, that, she, "", doesn't, want, to, put, more, chemical, in, her, ., "", she, do, not, follow, na-restrict, diet, ., in, the, past, week, ,, she, note, that, she, have, be, have, worsen, abd, distension, and, discomfort, ., she, denies, edema, ,, or, shortness of breath, ,, or, orthopnea, ., she, denies, ...]","[hepatitis c virus, cirrhosis, c/b, ascites, hiv, antiretroviral therapy, h/o, intravenous drug use, chronic obstructive pulmonary disease, bioplar, post-traumatic stress disorder, present, outside hospital, emergency department, worsen, abd, distension, past, week, pt, report, self-discontinue, lasix, spirnolactone, week, ago, feel, like, anything, want, put, chemical, fo

In [35]:
# View full list (long print)
pprint.pprint(df_extract["tokens_final"].iloc[0])

['hepatitis c virus',
 'cirrhosis',
 'c/b',
 'ascites',
 'hiv',
 'antiretroviral therapy',
 'h/o',
 'intravenous drug use',
 'chronic obstructive pulmonary disease',
 'bioplar',
 'post-traumatic stress disorder',
 'present',
 'outside hospital',
 'emergency department',
 'worsen',
 'abd',
 'distension',
 'past',
 'week',
 'pt',
 'report',
 'self-discontinue',
 'lasix',
 'spirnolactone',
 'week',
 'ago',
 'feel',
 'like',
 'anything',
 'want',
 'put',
 'chemical',
 'follow',
 'na-restrict',
 'diet',
 'past',
 'week',
 'note',
 'worsen',
 'abd',
 'distension',
 'discomfort',
 'denies',
 'edema',
 'shortness of breath',
 'orthopnea',
 'denies',
 'fever , chill , nausea , and vomit',
 'discharge or discontinue',
 'dysuria',
 'food',
 'poison',
 'week',
 'ago',
 'eat',
 'stale',
 'cake',
 'n/v',
 '20',
 'min',
 'food',
 'ingestion',
 'resolve',
 'day',
 'denies',
 'recent',
 'illness',
 'sick',
 'contact',
 'note',
 'notice',
 'gum',
 'bleeding',
 'brushing',
 'teeth',
 'recent',
 'week',
 

In [36]:
df_extract.to_csv("lda_preprocess.csv", index=False)

from google.colab import files
files.download('lda_preprocess.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>